# 5.Método Cadena Markov-Monte Carlo (MCMC)

In [1]:
using LinearAlgebra
using BenchmarkTools
using SparseArrays
using Random

using Plots
using PyCall
using DataFrames

## Implementación de la función que construye el sistema de ecuaciones de prueba $Ax = b$

In [2]:
function matriz_dispersa(n)
    e = ones(n)
    n2 =Int(n/2)
    diags = [-1,0,1]
    A = Matrix(spdiagm(-1 => -ones(n-1)
        ,0 => 3*ones(n),1 => -ones(n-1)))
    c = spdiagm(0 => ones(n)/2)
    ab = [x for x=1:n]
    ba = [(n+1)-x for x=1:n]
    c = Matrix(permute(c, ba, ab))

    A = A + c
    A[n2+1,n2] = -1
    A[n2,n2+1] = -1
    
    b = zeros(n,1)
    b[1] = 2.5
    b[n] = 2.5
    b[2:n-1] .= [1.5]
    b[n2:n2+1] .= [1]
    return A,b
end

matriz_dispersa (generic function with 1 method)

### Evaluación de parámetros

In [3]:

function preparametros(A,b,ϵ,δ)
    M = diagm(0 => diag(A))
    N = M-A
    T = inv(M) * N
    f = inv(M) * b
    nT, mT = size(T);
    print(nT, mT )

    S = fill(0, nT)
    P = fill(0., nT, mT)
    Pa = P
    [S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0]
    [P[i,j]= 1/S[i] for i in 1:nT, j in 1:mT if T[i,j] != 0 ]
    Pa = [accumulate(+, P[i, 1:mT]) for i in 1:nT]
    Pi = [1/nT for i in 1:nT];
    Nc = floor((0.6745/δ)^2*((norm(f)^2)/(1-norm(T))^2)) + 1
    return Nc, mT, f, Pa, T, P
end

preparametros (generic function with 1 method)

### Matriz de probabilidad acumulada pre-build con envío de parámetros, type-anotations y view

In [4]:
function mcmc_acc_par_ta(ϵ, Nc, mT, f, Pa, T, P)
    Xs = fill(0., mT)
    for i in 1:mT
        W_0 = 1.0
        for s in 1:Nc
            W = W_0; point = i; X = W_0 * f[i]::Float64
            while abs(W) >= ϵ
                nextpoint  = 1::Int64
                u = rand()
                while u >= Pa[point][nextpoint]::Float64
                    nextpoint = nextpoint + 1::Int64
                end
                    W_new = W *(T[point, nextpoint]/P[point, nextpoint])::Float64
                    X = X + W_new * f[nextpoint]::Float64
                point = nextpoint::Int64
                W = W_new::Float64
            end
        Xs[i] += X::Float64
        end
    end
    Xs = Xs/Nc::Float64
end

mcmc_acc_par_ta (generic function with 1 method)

In [19]:
n = [6, 10, 30, 50]
A,b = matriz_dispersa(n[1])
As = sparse(A)
bs = sparse(b)
ϵ = 0.1
δ = 0.1 
(Nc, mT, f, Pa, T, P) = preparametros(A,b,ϵ,δ)
(Ncs, mTs, fs, Pas, Ts, Ps) = preparametros(As,bs,ϵ,δ)

6666

(8623.0, 6, [0.8333333333333333; 0.5; … ; 0.5; 0.8333333333333333], [[0.0, 0.5, 0.5, 0.5, 0.5, 1.0], [0.3333333333333333, 0.3333333333333333, 0.6666666666666666, 0.6666666666666666, 1.0, 1.0], [0.0, 0.5, 0.5, 1.0, 1.0, 1.0], [0.0, 0.0, 0.5, 0.5, 1.0, 1.0], [0.0, 0.3333333333333333, 0.3333333333333333, 0.6666666666666666, 0.6666666666666666, 1.0], [0.5, 0.5, 0.5, 0.5, 1.0, 1.0]], [0.0 0.3333333333333333 … 0.0 -0.16666666666666666; 0.3333333333333333 0.0 … -0.16666666666666666 0.0; … ; 0.0 -0.16666666666666666 … 0.0 0.3333333333333333; -0.16666666666666666 0.0 … 0.3333333333333333 0.0], [0.0 0.5 … 0.0 0.5; 0.3333333333333333 0.0 … 0.3333333333333333 0.0; … ; 0.0 0.3333333333333333 … 0.0 0.3333333333333333; 0.5 0.0 … 0.5 0.0])

In [29]:
(A == As)


true

In [30]:
((Nc, mT, f, Pa, P) == (Ncs, mTs, fs, Pas, Ps))

true

In [31]:
@benchmark mcmc_acc_par_ta($ϵ, $Nc, $mT, $f, $Pa, $T, $P)

BenchmarkTools.Trial: 815 samples with 1 evaluation.
 Range (min … max):  5.426 ms …   8.553 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.070 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.130 ms ± 492.730 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂▂█▆▆▄▇▆▅█▃ ▄▁▃▇▄▅▄▅▃▃▂ ▆▁ ▂ ▁ ▁                             
  ████████████████████████████▇█▅█▇▅▆▅▄▄▄▄▃▄▄▃▃▂▃▂▁▂▁▁▁▂▂▁▁▁▂ ▅
  5.43 ms         Histogram: frequency by time        7.81 ms <

 Memory estimate: 256 bytes, allocs estimate: 2.

In [32]:
@benchmark mcmc_acc_par_ta($ϵ, $Ncs, $mT, $fs, $Pas, $Ts, $Ps)

BenchmarkTools.Trial: 786 samples with 1 evaluation.
 Range (min … max):  5.434 ms …   8.580 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.295 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.352 ms ± 519.750 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▂  ▁▄▄▂▂▄▆▇▄▆▄█▅█▃▆▄▆▂█▄▅▃ ▅▂ ▁▁ ▂                       
  ▆▆▇███▅█████████████████████████████▆██▇▆█▆▄▄▅▂▅▆▄▆▄▄▃▃▂▄▁▃ ▆
  5.43 ms         Histogram: frequency by time        7.73 ms <

 Memory estimate: 256 bytes, allocs estimate: 2.

In [33]:
X  = @btime mcmc_acc_par_ta($ϵ, $Nc, $mT, $f, $Pa, $T, $P)
Xs = @btime mcmc_acc_par_ta($ϵ, $Ncs, $mTs, $fs, $Pas, $Ts, $Ps)

  5.418 ms (2 allocations: 256 bytes)
  5.422 ms (2 allocations: 256 bytes)


6-element Array{Float64,1}:
 1.0119271144615298
 1.0117101002619389
 0.9954939883610078
 0.9954925367610096
 1.0018438219851056
 0.9970250949425885

In [34]:
X 

6-element Array{Float64,1}:
 0.9954057589200687
 1.0008335415285656
 1.004245618461406
 1.000210336176775
 0.998142729172908
 0.9975253998183536

In [35]:
norm(b-A*X)

0.021564596617558874

In [36]:
norm(bs-As*Xs)

0.04749927176942902